# Assignment 1 : Secondary structure prediction <br>
In this project we are going to perform protein secondary structure (PSS) prediction and protein family prediction on 2 datasets.<br>
Both these predictions are done using suport vector machines (SVM) and validated using cross validation. <br>

We are using the sklearn implementation for accuracy metrices and implementation for the SVM.


In [1]:
import numpy as np
import string
import os
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report , confusion_matrix
from sklearn.model_selection import KFold
np.random.seed(123)

### Our global variables <br>
The code below defines the different values for secondary structures, amino acids and protein families. <br>
It also contains 2 different hyper parameters, the first being the neighboorhood size, this defines how many neighboors are considered in the feature extraction. <br>
The second hyper parameter defines the max iterations, the suport vector machine may train. While the suport vector. <br>

The name of the output file that has to be written to can also be configured here. <br>


In [2]:
frequencies = np.zeros([3,20])
labels = [ 'Beta', 'Coil' , 'Helix']
natural_acid = ['ALA' , 'ARG', 'ASN', 'ASP', 'CYS' ,'GLN', 'GLU', 'GLY','HIS','ILE','LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
families = ["Beta", "Alpha","Alpha/beta"]
outputfile = "output.txt"
neighboors = 10 #for sliding windows, neighboors at both directions
Max_iterations = 50

### Defining functions to read in the data

The code below defines functions for reading the different kinds of data files.<br>
The family data file contains a list of protein id's allong with their corresponding protein family. <br>
This is returned as a dictionary, with as keys the pdb code of that protein. <br>


The PSS data contains protein sequences that were also found in the family data, here every amino acid of that sequence is listed allong with its corresponding secondary structure. <br>
The dataset is returned as input and target set to be preprocessed later, another dictionary is also returned that contains the secondary structures and takes as key the PDB code. 


In [3]:
#loading data

def loadFamily_data():
    file = open("cath_info.txt","r")
    PDB_TO_FAMILY = {}
    for line in file:
        line = line.replace("\n", "")
        pdb, chain, family = line.split("\t")
        if family in families:
            PDB_TO_FAMILY[pdb] = family
    return PDB_TO_FAMILY
    
def loadPSS_data(filename):
    data_x = []
    data_y = []
    secondaryStructure = {}
    file = open(  filename,"r")
    last_pdb = ""
    x_sequence = []
    y_sequence = []
    for line in file:
        line = line.replace("\n", "")
        pdb, chain, sequence_number,residue, secondary_structure = line.split("\t")
        if pdb == last_pdb and residue in natural_acid:
            x_sequence.append(residue)
            y_sequence.append(secondary_structure)
            if secondary_structure in labels:
                index_label = labels.index(secondary_structure)
            else: 
                index_label = 1
            frequencies[index_label, natural_acid.index(residue)] += 1
        elif  residue in natural_acid:
            data_y.append(y_sequence)
            data_x.append(x_sequence)
            x_sequence = [residue]
            y_sequence = [secondary_structure]
            secondaryStructure[pdb] = y_sequence
        last_pdb = pdb
    return data_x[1:] , data_y[1:], secondaryStructure

## defining our preprocessing

The feature extraction for PSS prediction is inspired by the paper "A protein secondary structure prediction framework based on the Extreme Learning Machine".
https://www.sciencedirect.com/science/article/pii/S0925231208001094 <br>
Here they used a one hot vector Vi, which we multiply by a so called statistics vector. This vector combines the probability of the structure occuring for that residue. And also the probability of an amino acid appearing in the same structure. <br>

The feature size of each residue is 23, the first 20 represent the statistics vector. Whereas the last 3 last features contain the self information for each secondary structure. <br>

The features that are used for the protein family prediction, are the frequencies of each secondary structure within a single protein sequence. The feature size is 3.

In [4]:
# This function computes the self information given a residue and secondary structure.
# input:
# res : string 
# struct : string
# output : float
def self_information(struct, res):
    res = natural_acid.index(res)
    if struct in labels:
        struct = labels.index(struct)
    else: 
        struct = 1
    val = (frequencies[struct,res] / np.sum(frequencies[:,res]))/ (np.sum(frequencies[struct,:]) / np.sum(frequencies))
    information = np.log(val)
    return information


# This function computes extra statistics features
# input:
# res : int 
# struct : int
# output : float
def extra_statistics_features(struct,res):
    Ni_x = frequencies[struct,res]
    Ni = np.sum(frequencies[:,res])
    Min_val = np.min(frequencies[:,res])
    val =  ( Ni_x / Min_val) * (Ni_x/Ni) 
    return val


# This creates feature vector given a residue
# input:
# residue : string 
# output : array([23])
def process_residue(residue):
    informations = [  self_information(struct, residue)  for struct in labels ]
    GORI = np.argmax(informations)
    if residue in natural_acid:
            Residue=natural_acid.index(residue)
    else:
            Residue= 0
    res = np.zeros([20+3])
    res [Residue] =extra_statistics_features(GORI,Residue)
    for i in range (0,len(labels)):
        res[20+i] = informations[i]
    return res


# This function replaces secondary structures, by the corresponding integer number.
# input:
# structures: list
# output : list
def preprocess_structure(structures):
    structures_ = structures[:]
    for i,  y in enumerate( structures_):
        if y in labels:
            structures_[i] = labels.index(y)
        else: 
            structures_[i] = 1
    return structures_


# This function takes as input a complete dataset and performs the preprocessing on it.
# input:
# x : list 
# y : list
# output : list , list
def preprocess_PSS_data(x,y):
    X,Y = x[:] , y[:]
    Data_X,Data_Y= [] , []
    for i,x in enumerate(X):
        y = Y[i]
        Data_X.append([process_residue(r) for r in x])
        Data_Y.append(preprocess_structure(y))
    return Data_X,Data_Y




## creating datasets for machine learning <br>
The last block discussed the preproccessing to extract features for both datasets. Here below we will define 2 functions that return us a dataset that is ready to be used as input for the suport vector machine. <br>

The function createPSS_Dataset goes over every every residue and appends the information of its neighboors to the sample.  

In [5]:
# This function creates new samples by combining the current residue, and its neighboors.
# input:
# x : list 
# y : list
# output : list,list
def createPSS_Dataset(x,y):
    X_Data = []
    Y_Data = []
    for i, sequnce in enumerate(x):
        for j, residue in enumerate(sequnce):
            prev = np.zeros([23*neighboors]).tolist()
            if j >= neighboors:
                prev = []
                for k in range(0,neighboors):
                    prev.extend(sequnce[j-k].tolist())
            next = np.zeros([23*neighboors]).tolist()
            if j < len(sequnce) - neighboors:
                next = []
                for k in range(0,neighboors):
                    next.extend(sequnce[j+k].tolist())
            x_sample = prev
            x_sample.extend(residue.tolist())
            x_sample.extend(next)
            y_sample = y[i][j]
            X_Data.append(x_sample)
            Y_Data.append(y_sample)
    return X_Data,Y_Data

# This function generates the dataset for family prediction.
# input:
# dict_ss : dictionary 
# dict_fam : dictionary
# output : array, array
def createFamily_dataset(dict_ss, dict_fam):
    x = []
    y = []
    for key in dict_ss:
        sequence= dict_ss[key]
        if key in dict_fam:
            x_feature = np.zeros([3])
            for SS in sequence: 
                if type(SS) is str and SS in labels:
                    SS = labels.index(SS)
                elif type(SS) is str and SS not in labels:
                    SS = 1
                x_feature[SS] += 1
            x_feature = x_feature/np.sum(x_feature)
            x.append(x_feature)
            y.append(families.index(dict_fam[key]))
    return np.asarray(x),  np.asarray(y)

### define accuracy measurements
Below we define some functions for calculating and discplaying different accuracy measurements. It contains a MCC and Q1 score for each classification aswell as Q3 score.

In [6]:
#Prints out the validation metrices for a given model
def printResult(y_test,X_test, model,family=False):
    matrix = confusion_matrix(y_test, model.predict(X_test))
    print(classification_report(y_test, model.predict(X_test)))
    q3,mcc =printStats(matrix)
    return matrix ,q3,mcc

def printStats(matrix,family=False):
    i = 0
    MCC = MCCscore(matrix)
    Q1 = Q1Score(matrix)
    for mcc , q1 in zip ( MCC , Q1):
        l = labels[i]
        if family:
            l= families[i]
        i+=1
        print("The MCC score for " + l + " equal " + str(mcc))
        print("The Q1 score for " + l + " equal " + str(q1))
    print("mean MCC score = " + str(np.mean(MCC)))
    print("mean Q1 score = " + str(np.mean(Q1)))
    correct = matrix[1][1] +matrix[0][0] + matrix[2][2]
    q3 = correct / np.sum(matrix)
    print ("Q3 score = " + str(q3))
    return q3, np.mean(MCC)


def Q1Score (matrix):
    res = np.zeros(matrix.shape[0])
    for i in range (0,len(res)):
        total = np.sum(matrix[i])
        correct = matrix[i][i]
        res[i] += float(correct)/float(total)
    return res

def MCCscore(matrix):
    res = np.zeros(matrix.shape[0])
    for i in range (0,len(res)):
        TP = matrix[i][i]
        TN = np.sum ( np.delete (np.delete(matrix, i, 0), i, 1))
        FP = np.sum(matrix[:,i])-TP
        FN = np.sum(matrix[i,:]) - TP
        mcc = (TP*TN - FP * FN) / np.sqrt( (TP+FP) * (TP+FN) * (TN+FP) * (TN+FN) )
        res[i] += mcc
    return res
        


## Cross validation <br>
The code below is used to split the dataset in multiple sets used for cross validation.  

In [7]:
def create_K_folds(x,y,k=10):
    folds = []
    length = len(y)
    interval = round(length/(k-1))
    for i in range (0,length,interval):
        train_x_,train_y_ = x[:i] + x[i+interval:] , y[:i] + y[i+interval:]
        test_x_, test_y_ = x[i:i+interval], y[i:i+interval]
        fold = [train_x_[:],train_y_[:],test_x_[:], test_y_[:]]
        folds.append(fold)
    if i+interval < length:
        train_x_, train_y_ = x[:i+interval] , y[:i+interval] 
        test_x_ , test_y_ = x[i+interval:] , y[i+interval:]
        fold = [train_x_[:],train_y_[:],test_x_[:], test_y_[:]]
        folds.append(fold)
    return folds

def create_K_folds_fam(x,y,k=10):
    folds = []
    length = len(y)
    interval = round(length/(k-1))
    for i in range (0,length,interval):
        train_x_ = []
        train_x_.extend(x[:i].tolist())
        train_x_.extend( x[i+interval:].tolist())
        train_y_=[]
        train_y_.extend( y[:i].tolist() )
        train_y_.extend(y[i+interval:].tolist())
        test_x_, test_y_ = x[i:i+interval], y[i:i+interval]
        fold = [train_x_[:],train_y_[:],test_x_[:], test_y_[:]]
        folds.append(fold)
    if i+interval < length:
        train_x_, train_y_ = x[:i+interval] , y[:i+interval] 
        test_x_ , test_y_ = x[i+interval:] , y[i+interval:]
        print(train_y_[:])
        fold = [train_x_[:],train_y_[:],test_x_[:], test_y_[:]]
        folds.append(fold)
    return folds

## Predicting using DSSP dataset
In this section we will compute the results for DSSP dataset, and in the end of the notebook these will be discussed. 

### predicting PSS

In [8]:
residue_sequences, structures , SS_dict = loadPSS_data("dssp_info.txt")
folds = create_K_folds(residue_sequences, structures)

In [9]:
SVC_struct = LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
CF_matrix = np.zeros([3,3])
for train_x,train_y,test_x, test_y in folds:
    X_train, Y_train = createPSS_Dataset(*preprocess_PSS_data(train_x, train_y))
    X_test, Y_test = createPSS_Dataset(*preprocess_PSS_data(test_x, test_y))
    SVC_struct.fit(X_train,Y_train)
    matrix = confusion_matrix(Y_test, SVC_struct.predict(X_test))
    CF_matrix+= matrix
    SVC_struct = LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
SVC_struct.fit(X_train,Y_train)  
results_PSS_DSSP = CF_matrix

## predicting the protein family

In [10]:
fam_dict = loadFamily_data()
x_fam , y_fam = createFamily_dataset(SS_dict, fam_dict)
folds_fam = create_K_folds_fam(x_fam , y_fam )

In [11]:
SVC_fam =LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
CF_matrix = np.zeros([3,3])
for X_train, Y_train,X_test, Y_test in folds_fam:
    SVC_fam.fit(X_train,Y_train)
    matrix = confusion_matrix(Y_test, SVC_fam.predict(X_test))
    CF_matrix+= matrix
    SVC_fam = LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
SVC_fam.fit(X_train,Y_train)    
results_FAM_DSSP = CF_matrix

## Predicting using STRIDE dataset
In this section we will compute the results for STRIDE dataset, and in the end of the notebook these will be discussed. 

### predicting PSS

In [12]:
residue_sequences, structures , SS_dict = loadPSS_data("stride_info.txt")
folds = create_K_folds(residue_sequences, structures)

In [13]:
SVC_struct = LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
CF_matrix = np.zeros([3,3])
for train_x,train_y,test_x, test_y in folds:
    X_train, Y_train = createPSS_Dataset(*preprocess_PSS_data(train_x, train_y))
    X_test, Y_test = createPSS_Dataset(*preprocess_PSS_data(test_x, test_y))
    SVC_struct.fit(X_train,Y_train)
    matrix = confusion_matrix(Y_test, SVC_struct.predict(X_test))
    CF_matrix+= matrix
    SVC_struct = LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
SVC_struct.fit(X_train,Y_train)  
results_PSS_STRIDE = CF_matrix

## predicting the protein family

In [14]:
fam_dict = loadFamily_data()
x_fam , y_fam = createFamily_dataset(SS_dict, fam_dict)
folds_fam = create_K_folds_fam(x_fam , y_fam )

In [15]:
SVC_fam =LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
CF_matrix = np.zeros([3,3])
for X_train, Y_train,X_test, Y_test in folds_fam:
    SVC_fam.fit(X_train,Y_train)
    matrix = confusion_matrix(Y_test, SVC_fam.predict(X_test))
    CF_matrix[:matrix.shape[0], :matrix.shape[1]] += matrix
    SVC_fam = LinearSVC(C=0.5, penalty='l1', max_iter=Max_iterations, dual=False)
SVC_fam.fit(X_train,Y_train)    
#q3,mcc = printStats(CF_matrix, family = True)
#print(CF_matrix)
results_FAM_STRIDE = CF_matrix

# discussing results  <br> 
We will start by discussing the general accuracy metrices for the entire dataset and afterwards we will evaluate the performance on individual samples. 

In [16]:
results = [results_PSS_DSSP,results_FAM_DSSP, results_PSS_STRIDE ,results_FAM_STRIDE ]
print(" Protein secondary structure prediction results on DSSP dataset \n ")
q3,mcc = printStats(results_PSS_DSSP)
print(results_PSS_DSSP)
print(" Protein secondary structure prediction results on STRIDE dataset \n ")
q3,mcc = printStats(results_PSS_STRIDE)
print(results_PSS_STRIDE)

 Protein secondary structure prediction results on DSSP dataset 
 
The MCC score for Beta equal 0.3492467337012902
The Q1 score for Beta equal 0.37695934752134574
The MCC score for Coil equal 0.42378457524430185
The Q1 score for Coil equal 0.7142237785559447
The MCC score for Helix equal 0.42206223918078845
The Q1 score for Helix equal 0.6639509558764217
mean MCC score = 0.3983645160421268
mean Q1 score = 0.5850446939845707
Q3 score = 0.6214294836320293
[[ 5916.  5350.  4428.]
 [ 2315. 21416.  6254.]
 [ 2082.  6250. 16462.]]
 Protein secondary structure prediction results on STRIDE dataset 
 
The MCC score for Beta equal 0.34669370565550095
The Q1 score for Beta equal 0.37998513747832546
The MCC score for Coil equal 0.41693965850319437
The Q1 score for Coil equal 0.6932161494095029
The MCC score for Helix equal 0.42542248285758655
The Q1 score for Helix equal 0.6810764497410133
mean MCC score = 0.39635194900542725
mean Q1 score = 0.5847592455429472
Q3 score = 0.6175355517032401
[[ 6136

In [17]:
print(" Family prediction results on DSSP dataset \n ")
q3,mcc = printStats(results_FAM_DSSP, family = True)
print(results_FAM_DSSP)
print(" Family  prediction results on STRIDE dataset \n ")
q3,mcc = printStats(results_FAM_STRIDE, family = True)
print(results_FAM_STRIDE)

 Family prediction results on DSSP dataset 
 
The MCC score for Beta equal 0.751496750225385
The Q1 score for Beta equal 0.65625
The MCC score for Alpha equal 0.8827613459686616
The Q1 score for Alpha equal 0.8790322580645161
The MCC score for Alpha/beta equal 0.7622181902209267
The Q1 score for Alpha/beta equal 0.9629629629629629
mean MCC score = 0.7988254288049911
mean Q1 score = 0.8327484070091596
Q3 score = 0.8768898488120951
[[ 63.   0.  33.]
 [  0. 109.  15.]
 [  3.   6. 234.]]
 Family  prediction results on STRIDE dataset 
 
The MCC score for Beta equal 0.743322692933483
The Q1 score for Beta equal 0.6534653465346535
The MCC score for Alpha equal 0.8824035586845425
The Q1 score for Alpha equal 0.8682170542635659
The MCC score for Alpha/beta equal 0.7588071076412477
The Q1 score for Alpha/beta equal 0.9651162790697675
mean MCC score = 0.7948444530864244
mean Q1 score = 0.8289328932893288
Q3 score = 0.875
[[ 66.   0.  35.]
 [  0. 112.  17.]
 [  4.   5. 249.]]


### discussion
We see that the Q3 and MCC score are very similar between the two datasets. 
We can also see that for the PSS dataset the classification "Beta" performs by far the worse, this is known to be the most difficult to recognise in bio informatics. <br>

For the family classification we see that the overall accuracy is far higher, but also that Beta performs by a little worse than the other classifications.



## discussing single samples <br>
Below we will first define some functions for testing a single sample and writing the output to a file. 

In [18]:
labels_short = ['E','C','H']
natural_acid_short = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
def string_to_char_list(string):
    lst = []
    for c in string:
        lst.append(c)
    return lst
def short_structure_to_long_notation(lst):
    result = []
    for struct in lst:
        index = labels_short.index(struct)
        result.append(labels[index])
    return result
def short_residue_to_long_notation(lst):
    result = []
    for res in lst:
        index = natural_acid_short.index(res)
        result.append(natural_acid[index])
    return result
def index_to_short_structure(lst):
    result = []
    for index in lst:
        result.append(labels_short[index])
    return(result)
def test_single_sample(residue_sequence,real_secondary_structure,model):
    real_secondary_structure_long = short_structure_to_long_notation(real_secondary_structure)
    residue_sequence_long = short_residue_to_long_notation(residue_sequence)
    x_input, y_target = preprocess_PSS_data([residue_sequence_long],[real_secondary_structure_long])
    x_input, y_target = createPSS_Dataset(x_input, y_target)
    result = model.predict(x_input)
    cf = np.zeros([3,3])
    for struct_target, index_predict in zip(real_secondary_structure, result):
        index_target = labels_short.index(struct_target)
        cf[index_target,index_predict] +=1
    return cf, index_to_short_structure(result)
def add_line_file(filename, name, sequence, q3,mcc,  family):
    file_obj  = open(filename, "a+")
    sequence = ''.join(sequence)
    line = name+ "\t" + sequence + "\t" + str(q3) + "\t" + str(mcc) + "\t" + family +"\n"
    file_obj.write(line)

## Sample 1 : 1ava C <br>

Below we wil evaluate the accuracy metrices for a single sample when predicted using the SVM.<br>
We also do family prediction using similar sequences, retrieved from uniprot.org for protein family prediction.<br>



In [19]:
name = "1ava C" 
residue_sequence = string_to_char_list("ADPPPVHDTDGHELRADANYYVLSANRAHGGGLTMAP\
GHGRHCPLFVSQDPNGQHDGFPVRITPYGVAPSDKIIR\
LSTDVRISFRAYTTCLQSTEWHIDSELAAGRRHVITGPVKDPSPSGRENAFRIEKYSGAEVHEYKLMSCGDWCQD\
LGVFRDLKGGAWFLGATEPYHVVVFKKAPPA")
real_secondary_structure = string_to_char_list("CCCCECECCCCCECECCCEEEEEECCHHHCCCEEEEEECCEEEEEEEEECCCCCCCCCCE\
EEEECCCCCCCCECECCCCEEEEECCCCCCCCCCECEECCCCECCECEEECCCCCCCCCC\
CHHHCEEEEECECCCCCCEEEEEECCCEEECEEECCCCCCCCEEECCCCECCEEEEEECC\
C")
matrix,prediction = test_single_sample(residue_sequence,real_secondary_structure,SVC_struct)
q3,mcc = printStats(matrix)
long_sec_struct = {name:short_structure_to_long_notation(real_secondary_structure)}
family = {name:"Beta"}
x_input , y_input = createFamily_dataset(long_sec_struct, family)
prediction_family = SVC_fam.predict(x_input)
prediction_family = families[prediction_family[0]]
print("protein family was "+family[name]+ " and model predicted :" + prediction_family)
add_line_file(outputfile, name, prediction, q3, mcc , prediction_family )

The MCC score for Beta equal 0.26846544835723013
The Q1 score for Beta equal 0.3013698630136986
The MCC score for Coil equal 0.2656360343355284
The Q1 score for Coil equal 0.6470588235294118
The MCC score for Helix equal 0.016485992856579968
The Q1 score for Helix equal 0.3333333333333333
mean MCC score = 0.18352915851644616
mean Q1 score = 0.42725400662548124
Q3 score = 0.4972375690607735
protein family was Beta and model predicted :Beta


### prediction using similar sequences

In [20]:
blast_sequences = "LASTALSRSADPPPVHDTDGHELRADANYYVLSANRAHGGGLTMAPGHGRLCPLFVSQDP\
NGQHDGFPVRITPYGVAPSDKIIRLSTDVRISFRAYTTCLQSTEWHIDSELAAGRRHVIT\
GPVKDPSPSGRENAFRIEKYSGAEVHEYKLMSCGDWCQDLGVFRDLKGGAWFLGATEPYH\
VVVFKKAPPA","DPPPVHDTDGNELRADANYYVLPANRAHGGGLTMAPGHGRRCPLFVSQEADGQRDGLPVR\
IAPHGGAPSDKIIRLSTDVRISFRAYTTCVQSTEWHIDSELVSGRRHVITGPVRDPSPSG\
RENAFRIEKYSGAEVHEYKLMACGDSCQDLGVFRDLKGGAWFLGATEPYHVVVFKKAPPA",
"MSSRRVGLLLLSLLATTLTCSADPPPVHDTDGNELRTDANYYVLPANRAHGGGLTMAPGH\
GRRCPLFVSQEADGQRDGLPVRIAPHGGAPSDKIIRLSTDVRISFRAYTTCVQSTEWHID\
SELVSGRRHVITGPVRDPSPSGRENAFRIEKYSGAEVHEYKLMACGDSCQDLGVFRDLKG\
GAWFLGATEPYHVVVFKKAPPA","MSSRRVGLLLLSLLATTLSYSADPPPVHDTDGNELRADANYYILPANRAHGGGLTMAPGH\
GRRCPLFVSQEADGQRDGLPVRIAPHGGGAPSDKIIRLSTDVRISFRAYTTCVQSTEWHI\
DSELVSGRRHVITGPVRDPSPSGRENAFRIEKYSGAEVHEYKLMACGDSCQDLGVFRDLK\
GGAWFLGATEPYHVIVF","VRITPYGVAPSDKIIRLSTDVRISFRAYTTCLQSTEWHIDSELAAGRRHVITGPVKDPSP\
SGRENAFRIEKYSGAEVHEYKLMSCGDWCQDLGVFRDLKGGAWFLGATEPYHVVVFKKAP\
PA"
predictions_PSS_blast = []
predictions_FAM_blast = []
for s in blast_sequences:
    s = string_to_char_list(s)
    matrix,prediction = test_single_sample(s,['C' for i in range(0,len(s))],SVC_struct)
    long_sec_struct = {name:short_structure_to_long_notation(prediction)}
    x_input , y_input = createFamily_dataset(long_sec_struct, family)
    prediction_family = SVC_fam.predict(x_input)
    predictions_PSS_blast.append(prediction)
    predictions_FAM_blast.append(prediction_family) 
print("protein family was "+family[name]+ " and model predicted :" + families[int(np.mean(predictions_FAM_blast))])

protein family was Beta and model predicted :Alpha/beta


In [25]:
name = "1avm A" 
residue_sequence = string_to_char_list("AVYTLPELPYDYSALEPYISGEIMELHHDKHHKAYVDGANTALDKLAEARDKADFGAINKLEKDLAFNLAGHVNH\
SVFWKNMAPKGSAPERPTDELGAAIDEFFGSFDNMKAQFTAAATGIQGSGWASLVWDPLGKRINTLQFYDHQNNL\
PAGSIPLLQLDMWEHAFYLQYKNVKGDYVKSWWNVVNWDDVALRFSEARVA")
real_secondary_structure = string_to_char_list("CCCCCCCCCCCCCCCCCCCCHHHHHHHHHCHHHHHHHHHHHHHHHHHHHHHHCCCCCHHH\
HHHHHHHHHHHHHHHHHHHHCECCCCCCCCCCCHHHHHHHHHHHCCHHHHHHHHHHHHHC\
CCCCEEEEEEEECCCCEEEEEEEECCCECCCCCCEEEEEEECCHHHCHHHHCCCHHHHHH\
HHHHHECHHHHHHHHHHHCCC")
matrix,prediction = test_single_sample(residue_sequence,real_secondary_structure,SVC_struct)
q3,mcc = printStats(matrix)
long_sec_struct = {name:short_structure_to_long_notation(real_secondary_structure)}
family = {name:"Alpha/beta"}
x_input , y_input = createFamily_dataset(SS_dict, fam_dict)
prediction_family = SVC_fam.predict(x_input)
prediction_family = families[prediction_family[0]]
print("protein family was "+family[name]+ " and model predicted :" + prediction_family)
add_line_file(outputfile, name, prediction, q3, mcc , prediction_family )

The MCC score for Beta equal 0.30583742680051546
The Q1 score for Beta equal 0.4230769230769231
The MCC score for Coil equal 0.4844330006524664
The Q1 score for Coil equal 0.7352941176470589
The MCC score for Helix equal 0.4118010583715828
The Q1 score for Helix equal 0.6448598130841121
mean MCC score = 0.4006904952748549
mean Q1 score = 0.6010769512693647
Q3 score = 0.6467661691542289
protein family was Alpha/beta and model predicted :Beta


### prediction using similar sequences

In [26]:
blast_sequences = "LASTALSRSADPPPVHDTDGHELRADANYYVLSANRAHGGGLTMAPGHGRLCPLFVSQDP\
NGQHDGFPVRITPYGVAPSDKIIRLSTDVRISFRAYTTCLQSTEWHIDSELAAGRRHVIT\
GPVKDPSPSGRENAFRIEKYSGAEVHEYKLMSCGDWCQDLGVFRDLKGGAWFLGATEPYH\
VVVFKKAPPA","DPPPVHDTDGNELRADANYYVLPANRAHGGGLTMAPGHGRRCPLFVSQEADGQRDGLPVR\
IAPHGGAPSDKIIRLSTDVRISFRAYTTCVQSTEWHIDSELVSGRRHVITGPVRDPSPSG\
RENAFRIEKYSGAEVHEYKLMACGDSCQDLGVFRDLKGGAWFLGATEPYHVVVFKKAPPA",
"MSSRRVGLLLLSLLATTLTCSADPPPVHDTDGNELRTDANYYVLPANRAHGGGLTMAPGH\
GRRCPLFVSQEADGQRDGLPVRIAPHGGAPSDKIIRLSTDVRISFRAYTTCVQSTEWHID\
SELVSGRRHVITGPVRDPSPSGRENAFRIEKYSGAEVHEYKLMACGDSCQDLGVFRDLKG\
GAWFLGATEPYHVVVFKKAPPA","MSSRRVGLLLLSLLATTLSYSADPPPVHDTDGNELRADANYYILPANRAHGGGLTMAPGH\
GRRCPLFVSQEADGQRDGLPVRIAPHGGGAPSDKIIRLSTDVRISFRAYTTCVQSTEWHI\
DSELVSGRRHVITGPVRDPSPSGRENAFRIEKYSGAEVHEYKLMACGDSCQDLGVFRDLK\
GGAWFLGATEPYHVIVF","VRITPYGVAPSDKIIRLSTDVRISFRAYTTCLQSTEWHIDSELAAGRRHVITGPVKDPSP\
SGRENAFRIEKYSGAEVHEYKLMSCGDWCQDLGVFRDLKGGAWFLGATEPYHVVVFKKAP\
PA"
predictions_PSS_blast = []
predictions_FAM_blast = []
for s in blast_sequences:
    s = string_to_char_list(s)
    matrix,prediction = test_single_sample(s,['C' for i in range(0,len(s))],SVC_struct)
    long_sec_struct = {name:short_structure_to_long_notation(prediction)}
    x_input , y_input = createFamily_dataset(long_sec_struct, family)
    prediction_family = SVC_fam.predict(x_input)
    predictions_PSS_blast.append(prediction)
    predictions_FAM_blast.append(prediction_family) 
print("protein family was "+family[name]+ " and model predicted :" + families[int(np.mean(predictions_FAM_blast))])

protein family was Alpha/beta and model predicted :Alpha/beta
